In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
dataset=pd.read_csv("Social_Network_Ads (1).csv")

In [4]:
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [5]:
dataset.isnull().sum()

User ID            0
Gender             0
Age                0
EstimatedSalary    0
Purchased          0
dtype: int64

In [6]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [7]:
dataset=dataset.astype(int)

In [8]:
dataset.head()

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1


In [9]:
dataset=dataset.drop("User ID" ,axis=1)

In [10]:
dataset.head()

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1


In [11]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [12]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset[["Purchased"]]

In [13]:
indep.head()

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1


In [14]:
dep.head()

,Purchased
0,0
1,0
2,0
3,0
4,0


In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=1/3,random_state=0)

In [16]:
x_train,x_test,y_train,y_test

(     Age  EstimatedSalary  Gender_Male
 218   46            96000            0
 101   28            59000            1
 311   39            96000            1
 194   28            89000            1
 326   41            72000            1
 ..   ...              ...          ...
 323   48            30000            0
 192   29            43000            1
 117   36            52000            1
 47    27            54000            0
 172   26           118000            0
 
 [266 rows x 3 columns],
      Age  EstimatedSalary  Gender_Male
 132   30            87000            1
 309   38            50000            0
 341   35            75000            1
 196   30            79000            0
 246   35            50000            0
 ..   ...              ...          ...
 168   29           148000            1
 150   26            15000            0
 393   60            42000            1
 66    24            19000            1
 240   42           149000            1
 
 [134 rows 

In [22]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {'solver':['newton-cg', 'lbfgs', 'liblinear', 'saga'],
             'penalty':['l2']} 



grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 


In [25]:
grid.fit(x_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Users\rsarv\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [26]:
re=grid.cv_results_

In [27]:
grid_predictions=grid.predict(x_test)

In [28]:

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,grid_predictions)

In [29]:
print(cm)

[[79  6]
 [10 39]]


In [30]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [31]:

print(clf_report)

              precision    recall  f1-score   support

           0       0.89      0.93      0.91        85
           1       0.87      0.80      0.83        49

    accuracy                           0.88       134
   macro avg       0.88      0.86      0.87       134
weighted avg       0.88      0.88      0.88       134



In [32]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'penalty': 'l2', 'solver': 'liblinear'}: 0.8794289739855382


In [33]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9507803121248499

In [34]:

table=pd.DataFrame.from_dict(re)

In [35]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.015441,0.001023,0.019103,0.003070,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,2
1,0.023694,0.012060,0.019137,0.002825,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,2
2,0.009147,0.003100,0.018010,0.002767,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
3,0.004843,0.002099,0.009824,0.003950,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,2


In [36]:
age=float(input("Age:"))
Estimated_salary=float(input("Estimated_salary:"))
sex_male=int(input("Sex Male 0 or 1:"))

Age:40
Estimated_salary:800000
Sex Male 0 or 1:1


In [37]:
Future_Prediction=grid.predict([[age,Estimated_salary,sex_male]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]


In [38]:
import pickle

In [39]:
filename= "finalized_model_Grid_logisticReg_classification.sav"

In [40]:
pickle.dump(grid,open(filename,'wb'))

In [41]:
loaded_model = pickle.load(open("finalized_model_Grid_logisticReg_classification.sav",'rb'))


In [42]:
result= loaded_model.predict([[age,Estimated_salary,sex_male]])

In [43]:

result

array([1])